In [1]:
import sys

sys.path.append("..")

import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoConfig
from trl import GRPOConfig

from src.price_process import PriceProcessor
from src.model import Qwen2ForAction
from src.trainer import ActionGRPOTrainer
from src.rewards import compute_yield_policy1

d:\Program\miniconda3\envs\DL\lib\site-packages\transformers\utils\generic.py:496: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
d:\Program\miniconda3\envs\DL\lib\site-packages\transformers\utils\generic.py:353: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
d:\Program\miniconda3\envs\DL\lib\site-packages\transformers\utils\generic.py:353: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## data

In [2]:
pe_df = pd.read_csv("../data/000300_pe.csv", encoding="gbk")
index_df = pd.read_csv("../data/000300_price.csv", encoding="gbk")
bond_df = pd.read_csv("../data/10yr_bond_yield.csv", encoding="gbk")

In [3]:
pe_df["tradeDate"] = pd.to_datetime(pe_df["tradeDate"])
index_df["tradeDate"] = pd.to_datetime(index_df["tradeDate"])
bond_df["tradeDate"] = pd.to_datetime(bond_df["tradeDate"])

In [4]:
index_df = index_df.merge(pe_df, how="left", on="tradeDate")

In [5]:
index_df = index_df[index_df["tradeDate"] >= "2013-01-01"]
len(index_df)

2974

In [6]:
bond_df = bond_df[bond_df["tradeDate"] >= "2013-01-01"]
len(bond_df)

3031

In [7]:
overall_df = index_df.merge(bond_df, how="left", on="tradeDate")

In [8]:
prompt_len = 8
max_len = 16
pp = PriceProcessor(overall_df, ["EPValue", "yield"],
                    ["openIndex", "closeIndex"],
                    prompt_len=prompt_len,
                    max_len=max_len)

In [16]:
dataset = pp.rolling(window=max_len, step=13)

In [17]:
dataset = dataset.rename_columns({
    "openIndex": "open_price",
    "closeIndex": "close_price"
})
# dataset.set_format("pt")

In [15]:
dataset["close_price"].shape

torch.Size([228, 8])

## model

In [18]:
config = AutoConfig.from_pretrained("Qwen/Qwen2.5-0.5B")

In [19]:
config.update({
    # "hidden_size": 2,
    "vocab_size": 2,
    "num_generations": 16,
    "max_length": max_len,
    # "num_attention_heads": 1,
})

In [20]:
model = Qwen2ForAction(config).to("cuda")
# model

Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


In [21]:
trainer_config = GRPOConfig(
    do_train=True,
    num_generations=2,
    beta=0.0,
    temperature=1,
    max_prompt_length=prompt_len,
    max_completion_length=max_len - prompt_len,
    use_vllm=False,
    num_iterations=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
)

In [22]:
trainer = ActionGRPOTrainer(
    model,
    reward_funcs=compute_yield_policy1,
    args=trainer_config,
    train_dataset=dataset,
)

In [23]:
trainer.train()

Step,Training Loss
